In [31]:
import spacy
nlp = spacy.load("en_core_web_sm")
from bs4 import BeautifulSoup
import os, sys
import random
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
#from cStringIO import StringIO
from io import StringIO
import datetime
import subprocess
import mysql.connector
from mysql.connector import (connection)
from mysql.connector import errorcode
from dotenv import load_dotenv
load_dotenv()
from collections import Counter
import re
from mysql.connector import Error
import helper_functions
import pickle


def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr,  laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    fp.close()
    device.close()
    str = retstr.getvalue()
    retstr.close()
    return str

In [24]:
def cleantext(textl,blobid):
    #remove unwanted characters
    #remove distracting single quotes
    textl = re.sub('\'','',textl)
    #remove digits and words containing digits
    textl = re.sub('\w*\d\w*','',textl)
    #replace extra spaces with single space
    textl = re.sub(' +',' ',textl)
    #remove unwanted lines starting from special charcters
    textl = re.sub(r'\n: \'\'.*','',textl)
    textl = re.sub(r'\n!.*','',textl)
    textl = re.sub(r'^:\'\'.*','',textl)
    textl = re.sub("(http\S+)", "", textl)
    #remove non-breaking new line characters
    textl = re.sub(r'\n',' ',textl)
    #remove punctuation
    #textl = re.sub(r'[^\w\s]',' ',textl)
    textl = re.sub(r'[^\x00-\x7f]', ' ', textl)
   # with open(str(blobid)+".txt", "w") as text_file:
   #     text_file.write(textl)
    return textl

In [34]:
notext = 0
goodtext = 0
pdf_errors = 0
extraction_not_allowed = 0
connection = get_connection()
cursor = connection.cursor(buffered=True)
query = """select aid,f.blobid, filecontent 
           from myfiles f join myblobs b 
        on f.blobid=b.blobid where type_id=20 and f.blobid not in (select blobid
        from my_job_market_papers) limit 250"""
cursor.execute(query)
rows = cursor.fetchall()
for row in rows:
    print ("creating a file for blobid " + str(row[1]))
    filename = str(row[1]) + '.pdf'
    f = open(filename, 'wb')
    f.write(row[2])
    f.close()
    try: 
        textl = convert_pdf_to_txt(filename)
        #ltext = textl.translate(mpa)
    except AttributeError as e :
        print ("PDF error", e)
        pdf_errors += 1
        os.remove(filename)
        continue
    except :
        print ("PDF error no extraction allowed")
        extraction_not_allowed += 1
        #os.remove(filename)
        continue
    #os.remove(filename)
    if len(textl) == 0: 
            notext += 1
            print ("*** failed to parse, or no text " + str(row[1]) + " ****")
            os.remove(filename)
            continue
    os.remove(filename)
    textl = cleantext(textl, str(row[1]))
    jmptokens = nlp(textl)
    stop_words = spacy.lang.en.stop_words.STOP_WORDS
    jmptokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in jmptokens]  
    #remove stopwords, and exclude words less than 2 characters
    jmptokens = [word for word in jmptokens if word not in stop_words and len(word) > 2]
#    print(Counter(jmptokens).most_common(5))
    p = pickle.dumps(jmptokens)
    sql = """insert into my_job_market_papers 
        (blobid,stext,serialized_tokens) 
        values (%s,%s,%s) """
    val = (row[1], textl, p)
    cursor.execute(sql,val)
    connection.commit()
#closing database connection.
cursor.close()
connection.close()
print("MySQL connection is closed")



Connected to MySQL database... 
creating a file for blobid 1619513
creating a file for blobid 1642006
creating a file for blobid 1638096
creating a file for blobid 1599871
creating a file for blobid 1613316
creating a file for blobid 1642527
creating a file for blobid 1552433
creating a file for blobid 1626738
creating a file for blobid 1644878
creating a file for blobid 1643603
creating a file for blobid 1643788
creating a file for blobid 1583755
creating a file for blobid 1582301
creating a file for blobid 1598962
creating a file for blobid 1643804
creating a file for blobid 1641766
creating a file for blobid 1639928
creating a file for blobid 1643846
creating a file for blobid 1644403
creating a file for blobid 1643599
creating a file for blobid 1643364
creating a file for blobid 1606291
creating a file for blobid 1619229
creating a file for blobid 1645134
creating a file for blobid 1367710
creating a file for blobid 1644153
creating a file for blobid 1598690
creating a file for blo